In [424]:
import pandas as pd
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam


In [425]:
df = pd.read_csv("../heart.csv")

In [426]:
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,1025.000000,1025.000000,1025.000000,1025.000000,1025.00000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000,1025.000000
mean,54.434146,0.695610,0.942439,131.611707,246.00000,0.149268,0.529756,149.114146,0.336585,1.071512,1.385366,0.754146,2.323902,0.513171
std,9.072290,0.460373,1.029641,17.516718,51.59251,0.356527,0.527878,23.005724,0.472772,1.175053,0.617755,1.030798,0.620660,0.500070
min,29.000000,0.000000,0.000000,94.000000,126.00000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,48.000000,0.000000,0.000000,120.000000,211.00000,0.000000,0.000000,132.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,56.000000,1.000000,1.000000,130.000000,240.00000,0.000000,1.000000,152.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,275.00000,0.000000,1.000000,166.000000,1.000000,1.800000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.00000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [427]:
df.isna().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [428]:
#spliting data to target and feature
x = df.drop('target', axis= 1)
y = df['target']

print(x.shape , y.shape)

(1025, 13) (1025,)


In [429]:
#spliting data to test and train
X_train, X_test, y_train, y_test = train_test_split(x,y , test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape , y_test.shape)


(820, 13) (820,)
(205, 13) (205,)


In [430]:
#repare logistic regression
log_reg = LogisticRegression(random_state=42 , max_iter=1000)
log_reg.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [431]:
#prepare  SVM  model
SVM_Model = SVC(kernel='linear', random_state=42)
SVM_Model.fit(X_train , y_train)

,C,1.0
,kernel,'linear'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [432]:
#prepare KNN model
KNN_Model = KNeighborsClassifier(n_neighbors=2)
KNN_Model.fit(X_train, y_train)

,n_neighbors,2
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [433]:
#prepare decision tree model
DT_Model = DecisionTreeClassifier(random_state=42)
DT_Model.fit(X_train, y_train)

,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,42
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [434]:
#prepare Random Forest Tree model
RF_Model = RandomForestClassifier(n_estimators=7,random_state=42)
RF_Model.fit(X_train, y_train)

,n_estimators,7
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [435]:
#prepare Naive Bayes model
nb_Model = GaussianNB()
nb_Model.fit(X_train, y_train)

,priors,None
,var_smoothing,1e-09


In [436]:
#prepare ANN model
NN_Model = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),
    keras.layers.BatchNormalization(),

    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.BatchNormalization(),

    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.3),

    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(1, activation='sigmoid')
])

NN_Model.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = NN_Model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=32,
    validation_split=0.2,
    callbacks=[EarlyStopping(patience=15, restore_best_weights=True)],
    verbose=1
)


test_loss, test_acc = NN_Model.evaluate(X_test, y_test, verbose=0)
print(f"the acc is : {test_acc *100}% and the test loss is : {test_loss *100}%")


predictions = NN_Model.predict(X_test)
results = (predictions > 0.5).astype(int)


Epoch 1/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 6s 36ms/step - accuracy: 0.5899 - loss: 0.7765 - val_accuracy: 0.6646 - val_loss: 0.6168
Epoch 2/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7790 - loss: 0.4565 - val_accuracy: 0.7378 - val_loss: 0.5592
Epoch 3/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8430 - loss: 0.3762 - val_accuracy: 0.7500 - val_loss: 0.5379
Epoch 4/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8460 - loss: 0.3416 - val_accuracy: 0.7500 - val_loss: 0.5115
Epoch 5/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8460 - loss: 0.3479 - val_accuracy: 0.7744 - val_loss: 0.4990
Epoch 6/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8765 - loss: 0.3380 - val_accuracy: 0.7561 - val_loss: 0.4925
Epoch 7/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8567 - loss: 0.3257 - val_accuracy: 0.8232 - val_loss: 0.4699
Epoch 8/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8750 - loss: 0.3103 - val_accuracy: 0.

In [437]:
Models_List = [log_reg , SVM_Model , KNN_Model, DT_Model ,RF_Model , nb_Model , NN_Model]
Models_Name = ['log_reg','SVM' , 'KNN','DT_Model','RF_Model' , 'nb_Model' , 'NN_Model']
Models_Prediction = [ log_reg.predict(X_test) , SVM_Model.predict(X_test) ,
                      KNN_Model.predict(X_test) , DT_Model.predict(X_test),
                      RF_Model.predict(X_test) , nb_Model.predict(X_test), results]

In [438]:
#spliting data to target and feature For GA
x_GA = df[['sex', 'cp', 'chol', 'oldpeak']]
y_GA = df['target']
print(x.shape , y.shape)

(1025, 13) (1025,)


In [439]:
#spliting data to test and train for GA
X_train_GA, X_test_GA, y_train_GA, y_test_GA = train_test_split(x_GA,y_GA , test_size=0.2, random_state=42)
print(X_train_GA.shape, y_train_GA.shape)
print(X_test_GA.shape , y_test_GA.shape)


(820, 4) (820,)
(205, 4) (205,)


In [440]:
#prepare the logistic regression after GA
x_log_reg_GA = df[['sex', 'cp', 'trestbps', 'thalach', 'oldpeak', 'thal']]
y_log_reg_GA = df['target']
print(x.shape , y.shape)

#spliting data to test and train for GA
X_train_log_reg_GA, X_test_log_reg_GA, y_train_log_reg_GA, y_test_log_reg_GA = train_test_split(x_log_reg_GA,y_log_reg_GA , test_size=0.2, random_state=42)
print(X_train_log_reg_GA.shape, y_train_log_reg_GA.shape)
print(X_test_log_reg_GA.shape , X_test_log_reg_GA.shape)


#repare logistic regression for GA
log_reg_GA = LogisticRegression(random_state=42 , max_iter=1000)
log_reg_GA.fit(X_train_log_reg_GA, y_train_log_reg_GA)

(1025, 13) (1025,)
(820, 6) (820,)
(205, 6) (205, 6)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [441]:
#prepare the SVM after GA
x_SVM_GA = df[['sex', 'cp', 'thalach', 'exang', 'oldpeak', 'thal']]
y_SVM_GA = df['target']
print(x.shape, y.shape)

#spliting data to test and train for GA
X_train_SVM_GA, X_test_SVM_GA, y_train_SVM_GA, y_test_SVM_GA = train_test_split(x_SVM_GA, y_SVM_GA, test_size=0.2,random_state=42)
print(X_train_SVM_GA.shape, y_train_SVM_GA.shape)
print(X_test_SVM_GA.shape, y_test_SVM_GA.shape)

#prepare  SVM  model for GA
SVM_Model_GA = SVC(kernel='linear', random_state=42)
SVM_Model_GA.fit(X_train_SVM_GA, y_train_SVM_GA)

(1025, 13) (1025,)
(820, 6) (820,)
(205, 6) (205,)


,C,1.0
,kernel,'linear'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [442]:
x_KNN_GA = df[['sex', 'cp', 'fbs', 'thalach', 'oldpeak', 'ca', 'thal']]
y_KNN_GA = df['target']
print(x.shape , y.shape)

#spliting data to test and train for GA
X_train_KNN_GA, X_test_KNN_GA, y_train_KNN_GA, y_test_KNN_GA = train_test_split(x_KNN_GA,y_KNN_GA , test_size=0.2, random_state=42)
print(X_train_KNN_GA.shape, y_train_KNN_GA.shape)
print(X_test_KNN_GA.shape , y_test_KNN_GA.shape)

#prepare KNN model for GA
KNN_Model_GA = KNeighborsClassifier(n_neighbors=2)
KNN_Model_GA.fit(X_train_KNN_GA, y_train_KNN_GA)

(1025, 13) (1025,)
(820, 7) (820,)
(205, 7) (205,)


,n_neighbors,2
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [443]:
x_DT_GA = df[['age', 'fbs', 'thalach', 'oldpeak']]
y_DT_GA = df['target']
print(x.shape , y.shape)

#spliting data to test and train for GA
X_train_DT_GA, X_test_DT_GA, y_train_DT_GA, y_test_DT_GA = train_test_split(x_DT_GA,y_DT_GA , test_size=0.2, random_state=42)
print(X_train_DT_GA.shape, y_train_DT_GA.shape)
print(X_test_DT_GA.shape , y_test_DT_GA.shape)

#prepare decision tree model for GA
DT_Model_GA = DecisionTreeClassifier(random_state=42)
DT_Model_GA.fit(X_train_DT_GA, y_train_DT_GA)

(1025, 13) (1025,)
(820, 4) (820,)
(205, 4) (205,)


,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,42
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [444]:
x_RF_GA = df[['sex', 'cp', 'chol', 'exang', 'oldpeak']]
y_RF_GA = df['target']
print(x.shape , y.shape)

#spliting data to test and train for GA
X_train_RF_GA, X_test_RF_GA, y_train_RF_GA, y_test_RF_GA = train_test_split(x_RF_GA,y_RF_GA , test_size=0.2, random_state=42)
print(X_train_RF_GA.shape, y_train_RF_GA.shape)
print(X_test_RF_GA.shape , y_test_RF_GA.shape)


#prepare Random Forest Tree model for GA
RF_Model_GA = RandomForestClassifier(n_estimators=7,random_state=42)
RF_Model_GA.fit(X_train_RF_GA, y_train_RF_GA)

(1025, 13) (1025,)
(820, 5) (820,)
(205, 5) (205,)


,n_estimators,7
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [445]:
x_nb_GA = df[['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']]
y_nb_GA = df['target']
print(x.shape , y.shape)

#spliting data to test and train for GA
X_train_nb_GA, X_test_nb_GA, y_train_nb_GA, y_test_nb_GA = train_test_split(x_nb_GA,y_nb_GA , test_size=0.2, random_state=42)
print(X_train_nb_GA.shape, y_train_nb_GA.shape)
print(X_test_nb_GA.shape , y_test_nb_GA.shape)


#prepare Naive Bayes model for GA
nb_Model_GA = GaussianNB()
nb_Model_GA.fit(X_train_nb_GA, y_train_nb_GA)

(1025, 13) (1025,)
(820, 8) (820,)
(205, 8) (205,)


,priors,None
,var_smoothing,1e-09


In [446]:
x_ANN_GA = df[['age', 'cp', 'trestbps', 'restecg', 'exang', 'oldpeak', 'ca', 'thal']]
y_ANN_GA = df['target']
print(x.shape , y.shape)

#spliting data to test and train for GA
X_train_ANN_GA, X_test_ANN_GA, y_train_ANN_GA, y_test_ANN_GA = train_test_split(x_ANN_GA,y_ANN_GA , test_size=0.2, random_state=42)
print(X_train_ANN_GA.shape, y_train_ANN_GA.shape)
print(X_test_ANN_GA.shape , y_test_ANN_GA.shape)



#prepare ANN model  for GA
NN_Model_GA = keras.Sequential([
    keras.layers.Input(shape=(X_train_ANN_GA.shape[1],)),
    keras.layers.BatchNormalization(),

    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.BatchNormalization(),

    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.3),

    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(1, activation='sigmoid')
])

NN_Model_GA.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = NN_Model_GA.fit(
    X_train_ANN_GA, y_train_ANN_GA,
    epochs=200,
    batch_size=32,
    validation_split=0.2,
    callbacks=[EarlyStopping(patience=15, restore_best_weights=True)],
    verbose=1
)


test_loss, test_acc = NN_Model_GA.evaluate(X_test_ANN_GA, y_test_ANN_GA, verbose=0)
print(f"the acc is : {test_acc *100}% and the test loss is : {test_loss *100}%")


predictions = NN_Model_GA.predict(X_test_ANN_GA)
results_GA = (predictions > 0.5).astype(int)

(1025, 13) (1025,)
(820, 8) (820,)
(205, 8) (205,)
Epoch 1/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 6s 36ms/step - accuracy: 0.6372 - loss: 0.6970 - val_accuracy: 0.4817 - val_loss: 0.7688
Epoch 2/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8140 - loss: 0.4525 - val_accuracy: 0.4817 - val_loss: 0.8517
Epoch 3/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8232 - loss: 0.3974 - val_accuracy: 0.5244 - val_loss: 0.6882
Epoch 4/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8323 - loss: 0.3922 - val_accuracy: 0.6098 - val_loss: 0.6373
Epoch 5/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8384 - loss: 0.3708 - val_accuracy: 0.6280 - val_loss: 0.5931
Epoch 6/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8430 - loss: 0.3395 - val_accuracy: 0.6829 - val_loss: 0.5685
Epoch 7/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8399 - loss: 0.3428 - val_accuracy: 0.7378 - val_loss: 0.5243
Epoch 8/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step -

In [447]:
Models_List_GA = [log_reg_GA , SVM_Model_GA , KNN_Model_GA, DT_Model_GA ,RF_Model_GA , nb_Model_GA , NN_Model_GA]
Models_Name_GA = ['log_reg_GA','SVM_GA' , 'KNN_GA','DT_Model_GA','RF_Model_GA' , 'nb_Model_GA' , 'NN_Model_GA']
Models_Prediction_GA = [ log_reg_GA.predict(X_test_log_reg_GA) , SVM_Model_GA.predict(X_test_SVM_GA) ,
                      KNN_Model_GA.predict(X_test_KNN_GA) , DT_Model_GA.predict(X_test_DT_GA),
                      RF_Model_GA.predict(X_test_RF_GA) , nb_Model_GA.predict(X_test_nb_GA), results_GA]

In [448]:
results = []

for model, name, pred in zip(Models_List, Models_Name, Models_Prediction):
    accuracy = accuracy_score(y_test, pred) * 100
    precision = precision_score(y_test, pred, average='weighted', zero_division=0) * 100
    recall = recall_score(y_test, pred, average='weighted', zero_division=0) * 100
    f1 = f1_score(y_test, pred, average='weighted', zero_division=0) * 100

    results.append({
        'Model': name,
        'Accuracy ': f"{accuracy:.2f}",
        'Precision': f"{precision:.2f}",
        'Recall': f"{recall:.2f}",
        'F1-Score': f"{f1:.2f}"
    })

results_df = pd.DataFrame(results)

print("\n" + "="*70)
print("MODEL COMPARISON TABLE Before GA")
print("="*70)
print(results_df.to_string(index=False))



results = []

for model, name, pred in zip(Models_List_GA, Models_Name_GA, Models_Prediction_GA):
    accuracy = accuracy_score(y_test, pred) * 100
    precision = precision_score(y_test, pred, average='weighted', zero_division=0) * 100
    recall = recall_score(y_test, pred, average='weighted', zero_division=0) * 100
    f1 = f1_score(y_test, pred, average='weighted', zero_division=0) * 100

    results.append({
        'Model': name,
        'Accuracy ': f"{accuracy:.2f}",
        'Precision ': f"{precision:.2f}",
        'Recall': f"{recall:.2f}",
        'F1-Score': f"{f1:.2f}"
    })

results_df = pd.DataFrame(results)


print("\n" + "="*70)
print("MODEL COMPARISON TABLE After GA")
print("="*70)
print(results_df.to_string(index=False))


MODEL COMPARISON TABLE Before GA
   Model Accuracy  Precision Recall F1-Score
 log_reg     79.51     80.23  79.51    79.38
     SVM     80.49     81.68  80.49    80.29
     KNN     95.61     95.97  95.61    95.60
DT_Model     98.54     98.58  98.54    98.54
RF_Model     98.54     98.58  98.54    98.54
nb_Model     80.00     81.05  80.00    79.82
NN_Model     93.66     93.66  93.66    93.66

MODEL COMPARISON TABLE After GA
      Model Accuracy  Precision  Recall F1-Score
 log_reg_GA     82.44      82.63  82.44    82.41
     SVM_GA     82.44      82.74  82.44    82.39
     KNN_GA     99.02      99.04  99.02    99.02
DT_Model_GA    100.00     100.00 100.00   100.00
RF_Model_GA    100.00     100.00 100.00   100.00
nb_Model_GA     82.93      83.08  82.93    82.90
NN_Model_GA     89.27      89.51  89.27    89.25


In [449]:
def print_detailed_report(y_true, y_pred, model_name=""):

    accuracy = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)

    precision = precision_score(y_true, y_pred, average='binary')
    recall = recall_score(y_true, y_pred, average='binary')
    f1_score_val = f1_score(y_true, y_pred, average='binary')


    if cm.shape == (2, 2):
        TN, FP, FN, TP = cm.ravel()
        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
    else:
        TN = cm[0, 0] if cm.shape[0] == 2 else "N/A"
        FP = cm[0, 1] if cm.shape[0] == 2 else "N/A"
        FN = cm[1, 0] if cm.shape[0] == 2 else "N/A"
        TP = cm[1, 1] if cm.shape[0] == 2 else "N/A"
        specificity = "N/A"


    print("\n" + "="*70)
    print(f"DETAILED EVALUATION REPORT - {model_name}")
    print("="*70)


    print("\nBASIC METRICS")
    print("-"*40)
    print(f"{'Accuracy:':<25} {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"{'Precision:':<25} {precision:.4f}")
    print(f"{'Recall/Sensitivity:':<25} {recall:.4f}")
    print(f"{'Specificity:':<25} {specificity:.4f}")
    print(f"{'F1-Score:':<25} {f1_score_val:.4f}")


    print("\n CONFUSION MATRIX")
    print("-"*40)
    print(f"{'':<15} {'Predicted':^20}")
    print(f"{'':<15} {'No (0)':^10} {'Yes (1)':^10}")
    print("-"*45)
    print(f"{'Actual No (0)':<15} {TN:^10} {FP:^10}")
    print(f"{'Actual Yes (1)':<15} {FN:^10} {TP:^10}")


    print("\nCONFUSION MATRIX BREAKDOWN")
    print("-"*40)
    print(f"{'True Positive (TP):':<25} {TP:>4d} - Correctly predicted positive")
    print(f"{'True Negative (TN):':<25} {TN:>4d} - Correctly predicted negative")
    print(f"{'False Positive (FP):':<25} {FP:>4d} - False alarm (Type I Error)")
    print(f"{'False Negative (FN):':<25} {FN:>4d} - Missed case (Type II Error)")


    if isinstance(TP, int) and isinstance(TN, int):
        total = TP + TN + FP + FN
        print("\nRATES & PERCENTAGES")
        print("-"*40)
        print(f"{'Overall Accuracy Rate:':<25} {(TP + TN)/total:.2%}")
        print(f"{'Overall Error Rate:':<25} {(FP + FN)/total:.2%}")
        print(f"{'False Positive Rate:':<25} {FP/(FP + TN):.2%}" if (FP + TN) > 0 else f"{'False Positive Rate:':<25} 0.00%")
        print(f"{'False Negative Rate:':<25} {FN/(TP + FN):.2%}" if (TP + FN) > 0 else f"{'False Negative Rate:':<25} 0.00%")
        print(f"{'Positive Predictive Value:':<25} {precision:.2%}")
        print(f"{'Negative Predictive Value:':<25} {TN/(TN + FN):.2%}" if (TN + FN) > 0 else f"{'Negative Predictive Value:':<25} 0.00%")


    print("\nCOMPLETE CLASSIFICATION REPORT")
    print("-"*40)
    print(classification_report(y_true, y_pred, digits=4))


print("="*70)
print("MODEL COMPARISON AND EVALUATION")
print("="*70)

for model, name, pred in zip(Models_List, Models_Name, Models_Prediction):
    print_detailed_report(y_test, pred, name)

MODEL COMPARISON AND EVALUATION

DETAILED EVALUATION REPORT - log_reg

BASIC METRICS
----------------------------------------
Accuracy:                 0.7951 (79.51%)
Precision:                0.7563
Recall/Sensitivity:       0.8738
Specificity:              0.7157
F1-Score:                 0.8108

 CONFUSION MATRIX
----------------------------------------
                     Predicted      
                  No (0)    Yes (1)  
---------------------------------------------
Actual No (0)       73         29    
Actual Yes (1)      13         90    

CONFUSION MATRIX BREAKDOWN
----------------------------------------
True Positive (TP):         90 - Correctly predicted positive
True Negative (TN):         73 - Correctly predicted negative
False Positive (FP):        29 - False alarm (Type I Error)
False Negative (FN):        13 - Missed case (Type II Error)

COMPLETE CLASSIFICATION REPORT
----------------------------------------
              precision    recall  f1-score   support

 

In [450]:
def print_detailed_report(y_true, y_pred, model_name=""):

    accuracy = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)

    precision = precision_score(y_true, y_pred, average='binary')
    recall = recall_score(y_true, y_pred, average='binary')
    f1_score_val = f1_score(y_true, y_pred, average='binary')


    if cm.shape == (2, 2):
        TN, FP, FN, TP = cm.ravel()
        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
    else:
        TN = cm[0, 0] if cm.shape[0] == 2 else "N/A"
        FP = cm[0, 1] if cm.shape[0] == 2 else "N/A"
        FN = cm[1, 0] if cm.shape[0] == 2 else "N/A"
        TP = cm[1, 1] if cm.shape[0] == 2 else "N/A"
        specificity = "N/A"


    print("\n" + "="*70)
    print(f"DETAILED EVALUATION REPORT - {model_name}")
    print("="*70)


    print("\nBASIC METRICS")
    print("-"*40)
    print(f"{'Accuracy:':<25} {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"{'Precision:':<25} {precision:.4f}")
    print(f"{'Recall/Sensitivity:':<25} {recall:.4f}")
    print(f"{'Specificity:':<25} {specificity:.4f}")
    print(f"{'F1-Score:':<25} {f1_score_val:.4f}")


    print("\n CONFUSION MATRIX")
    print("-"*40)
    print(f"{'':<15} {'Predicted':^20}")
    print(f"{'':<15} {'No (0)':^10} {'Yes (1)':^10}")
    print("-"*45)
    print(f"{'Actual No (0)':<15} {TN:^10} {FP:^10}")
    print(f"{'Actual Yes (1)':<15} {FN:^10} {TP:^10}")


    print("\nCONFUSION MATRIX BREAKDOWN")
    print("-"*40)
    print(f"{'True Positive (TP):':<25} {TP:>4d} - Correctly predicted positive")
    print(f"{'True Negative (TN):':<25} {TN:>4d} - Correctly predicted negative")
    print(f"{'False Positive (FP):':<25} {FP:>4d} - False alarm (Type I Error)")
    print(f"{'False Negative (FN):':<25} {FN:>4d} - Missed case (Type II Error)")


    if isinstance(TP, int) and isinstance(TN, int):
        total = TP + TN + FP + FN
        print("\nRATES & PERCENTAGES")
        print("-"*40)
        print(f"{'Overall Accuracy Rate:':<25} {(TP + TN)/total:.2%}")
        print(f"{'Overall Error Rate:':<25} {(FP + FN)/total:.2%}")
        print(f"{'False Positive Rate:':<25} {FP/(FP + TN):.2%}" if (FP + TN) > 0 else f"{'False Positive Rate:':<25} 0.00%")
        print(f"{'False Negative Rate:':<25} {FN/(TP + FN):.2%}" if (TP + FN) > 0 else f"{'False Negative Rate:':<25} 0.00%")
        print(f"{'Positive Predictive Value:':<25} {precision:.2%}")
        print(f"{'Negative Predictive Value:':<25} {TN/(TN + FN):.2%}" if (TN + FN) > 0 else f"{'Negative Predictive Value:':<25} 0.00%")


    print("\nCOMPLETE CLASSIFICATION REPORT")
    print("-"*40)
    print(classification_report(y_true, y_pred, digits=4))


print("="*70)
print("MODEL COMPARISON AND EVALUATION after GA ")
print("="*70)

for model, name, pred in zip(Models_List_GA, Models_Name_GA, Models_Prediction_GA):
    print_detailed_report(y_test, pred, name)

MODEL COMPARISON AND EVALUATION after GA 

DETAILED EVALUATION REPORT - log_reg_GA

BASIC METRICS
----------------------------------------
Accuracy:                 0.8244 (82.44%)
Precision:                0.8018
Recall/Sensitivity:       0.8641
Specificity:              0.7843
F1-Score:                 0.8318

 CONFUSION MATRIX
----------------------------------------
                     Predicted      
                  No (0)    Yes (1)  
---------------------------------------------
Actual No (0)       80         22    
Actual Yes (1)      14         89    

CONFUSION MATRIX BREAKDOWN
----------------------------------------
True Positive (TP):         89 - Correctly predicted positive
True Negative (TN):         80 - Correctly predicted negative
False Positive (FP):        22 - False alarm (Type I Error)
False Negative (FN):        14 - Missed case (Type II Error)

COMPLETE CLASSIFICATION REPORT
----------------------------------------
              precision    recall  f1-score